In [1]:
import json
import time
from dotenv import load_dotenv
load_dotenv()
import sys
import os
from azure.identity import DefaultAzureCredential
from crm_store import CRMStore
from conversation_store import ConversationStore
from handlers import SemanticKernelHandler
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents import AgentGroupChat

from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent

from sk.agents.agent_group_chat import create_agent_group_chat

key = DefaultAzureCredential()
db = ConversationStore(
        url=os.getenv("COSMOSDB_ENDPOINT"),
        key=key,
        database_name=os.getenv("COSMOSDB_DATABASE_NAME"),
        container_name=os.getenv("COSMOSDB_CONTAINER_FSI_INS_USER_NAME"),
    )


### SK Agent Group Chat - Single Agent / multi skill test - no history single shot

In [2]:


handler = SemanticKernelHandler(db)

userId = 'sk_test_ins_1'
user_data = {'chat_histories': {}}
chat_history = ChatHistory()

#team_query_single = 'Provide the coverages of the ZTravel Plus policy'
team_query_single = 'Provide me the policies of my client John Doe in a table format'


# AgentGroupChat handling        
agent_group_chat = create_agent_group_chat(handler.kernel, chat_history)
#agent_group_chat.set_chat_history(chat_history)

await agent_group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=team_query_single))

# responses = list(reversed([item async for item in group_chat.invoke()]))
async for item in agent_group_chat.invoke():
    print("---------------- >>>>>>  -------------------")
    print(f"{item.name}: {item.content}")
    continue

response = list(reversed([item async for item in agent_group_chat.get_chat_messages()]))

print(f"SK AgentGroupChat reponder response = {response}")




CRM: full name: John Doe
---------------- >>>>>>  -------------------
Assistant: Here are the policies for the client John Doe:

| Policy No | Basic Plan Product | Product Type               | Policy Status | Effective Date        | Expiry Date          | Overseas Medical Expenses | Follow up Treatment in Malaysia | Alternative Medicine | Overseas Hospital Income | Child Care Benefit | Compassionate Visit | Emergency Medical Evacuation | Medical Repatriation | Repatriation of Mortal Remains |
|-----------|--------------------|----------------------------|---------------|-----------------------|----------------------|---------------------------|-------------------------------|---------------------|--------------------------|-------------------|--------------------|-----------------------------|----------------------|-------------------------------|
| A1234567  | P230ZII            | Ztravel International      | A             | 2024-06-18 00:00:00.000 | 2024-07-13 00:00:00.000 | 500,000 

In [3]:
def extract_sk_responses(response):
    """
    Extracts the assistant's response from a list of ChatMessageContent objects.

    Args:
        chat_messages (list): A list of ChatMessageContent objects.

    Returns:
        dict: A dictionary with keys 'role', 'name', and 'content' containing the assistant's response.
    """
    results = []
    for message in response:
        # Initialize 
        content = ''
        name = ''
        role = ''

        if message.role == AuthorRole.ASSISTANT:
            name = AuthorRole.ASSISTANT.value    
            role = AuthorRole.ASSISTANT.value   
        elif message.role == AuthorRole.USER:
            name = AuthorRole.USER.value    
        else:
            continue

        if not message.name == '':
            # Check if items are present
            if message.items and len(message.items) > 0:
                # Iterate over items to extract text content
                for item in message.items:
                    if hasattr(item, 'text') and item.text:
                        content += item.text
                    elif hasattr(item, 'result') and item.result:
                        content += item.result
            elif message.inner_content:
                # Use inner_content if items are not present
                content = message.inner_content
            else:
                # As a fallback, check for 'content' in metadata
                content = message.metadata.get('content', '')

            # Build the response dictionary
            result = {
                'role': role,
                'name': name,
                'content': content.strip()
            }
            results.append(result)
    return results



In [4]:
extract_sk_responses(response)

[{'role': '',
  'name': 'user',
  'content': 'Provide me the policies of my client John Doe in a table format'},
 {'role': 'assistant', 'name': 'assistant', 'content': ''},
 {'role': 'assistant',
  'name': 'assistant',
  'content': 'Here are the policies for the client John Doe:\n\n| Policy No | Basic Plan Product | Product Type               | Policy Status | Effective Date        | Expiry Date          | Overseas Medical Expenses | Follow up Treatment in Malaysia | Alternative Medicine | Overseas Hospital Income | Child Care Benefit | Compassionate Visit | Emergency Medical Evacuation | Medical Repatriation | Repatriation of Mortal Remains |\n|-----------|--------------------|----------------------------|---------------|-----------------------|----------------------|---------------------------|-------------------------------|---------------------|--------------------------|-------------------|--------------------|-----------------------------|----------------------|------------------

### SK Agent Group Chat - Single Agent / multi skill test - with History - and handler

In [ ]:

import logging

# Remove all handlers associated with the root logger object.
#for handler in logging.root.handlers[:]:
#    logging.root.removeHandler(handler)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Set the logger level for your module if necessary
logger = logging.getLogger('handlers')
logger.setLevel(logging.INFO)

handler = SemanticKernelHandler(db)

user_id = 'sk_test_ins_1'
if not db.read_user_info(user_id):
    user_data = {'chat_histories': {}}
    db.create_user(user_id, user_data)

user_data = db.read_user_info(user_id)

team_query_single = 'Provide me details about my client John Doe policies in a table format'


try:
    await handler.handle_request(user_id, None, team_query_single, False, '', user_data)
except Exception as e:
    logging.error(f"Error in handler: {e}")



In [8]:
#continuing the previous chat:
user_data = db.read_user_info(user_id)

team_query_single = 'Can he travel to Bali with his current coverages?'

try:
    await handler.handle_request(user_id, "20241205_726", team_query_single, False, '', user_data)
except Exception as e:
    logging.error(f"Error in handler: {e}")

2024-12-05 16:26:46,830 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://aga-gbb-ai-cosmosdb-swedencentral.documents.azure.com:443/dbs/rminsights/colls/user_fsi_ins_data/docs/'
Request method: 'POST'
Request headers:
    'Cache-Control': 'no-cache'
    'x-ms-version': 'REDACTED'
    'x-ms-documentdb-query-iscontinuationexpected': 'REDACTED'
    'x-ms-consistency-level': 'REDACTED'
    'x-ms-documentdb-isquery': 'REDACTED'
    'Content-Type': 'application/query+json'
    'x-ms-session-token': 'REDACTED'
    'x-ms-documentdb-query-enablecrosspartition': 'REDACTED'
    'Accept': 'application/json'
    'x-ms-cosmos-correlated-activityid': 'REDACTED'
    'Content-Length': '104'
    'User-Agent': 'azsdk-python-cosmos/4.5.1 Python/3.10.12 (Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.35)'
    'authorization': 'REDACTED'
A body is sent with the request
2024-12-05 16:26:47,097 - azure.core.pipeline.policies.http_logging_policy - INFO - Response st